In [ ]:
import os
import pandas as pd
import PIL
from PIL import Image
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch import nn


ROOT_PATH = ".."

DATA_PATH = os.path.join(ROOT_PATH,"input/rice-data/dataset")
IMAGES = os.path.join(DATA_PATH,"Images")


%ls $DATA_PATH
train_data = pd.read_csv(DATA_PATH+"/Train.csv")


In [ ]:
idx_to_drop=[]
for i,row in  enumerate(train_data.Image_id):
    if train_data.Image_id[i][-8:-4]=="_rgn":
        idx_to_drop.append(i)
        
train_data.drop(index=idx_to_drop, inplace=True)
train_data.reset_index(drop=True,inplace=True)

In [ ]:
import sys

def name2rgn(name_string):
    return name_string[:-4]+"_rgn.jpg"


def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False


class Identity_lay(nn.Module):
    def __init__(self):
        super(Identity_lay, self).__init__()
        
    def forward(self, x):
        return x           
            
def init_model(base,extract_feature):
    #vgg
    if base =="vgg":
        model_tmp = torch.hub.load('pytorch/vision:v0.10.0', 'vgg11', pretrained=True)
        train_batch_size=128
        set_parameter_requires_grad(model_tmp, extract_feature)
        num_ftrs = model_tmp.classifier[6].out_features
        model_tmp.classifier.append(nn.ReLU(inplace=True))
        model_tmp.classifier.append(nn.Dropout())
        model_tmp.classifier.append(nn.Linear(num_ftrs,NUM_CLASSES))
    
    elif base=="alex":        
        model_tmp = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
        train_batch_size=512
        
    elif base == "squeeze":
        model_tmp=model = torch.hub.load('pytorch/vision:v0.10.0', 'squeezenet1_0', pretrained=True)
    
    elif base=="ggNet":
        model_tmp = torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=True)
        num_ftrs = model_tmp.fc.in_features
#         model_tmp.fc=nn.Linear(num_ftrs,NUM_CLASSES)
        model_tmp.fc=Identity_lay()
        model_tmp.dropout=Identity_lay()
        train_batch_size=512

    return model_tmp


def params2update(model,extract_feature):
    print("Params to learn:")
    params_to_update=model.parameters()
    if extract_feature:
        params_to_update = []
        for name,param in model.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                print("\t",name)
    else:
        for name,param in model.named_parameters():
            if param.requires_grad == True:
                print("\t",name)
    return params_to_update

class mem_tracker:
    
    def __init__(self):
        self.last_mem = 0
    
    def get_mem(self):
        tmp=torch.cuda.memory_allocated()
        print("Total ",tmp/1e6,'MB ---',"Allocated from last check ",(tmp-self.last_mem)/(1e6),'MB')
        self.last_mem=tmp

In [ ]:
import cv2

#take 4 random images 
idx = torch.randperm(len(train_data))

plt.figure()

#subplot(r,c) provide the no. of rows and columns
fig, ax = plt.subplots(2 , 4 , figsize=(100,50))

for i in range(4):
    img_path = os.path.join(IMAGES,train_data.Image_id[idx[i].item()])
    img_path_rgn = os.path.join(IMAGES,train_data.Image_id[idx[i].item()])[:-4]+"_rgn.jpg"
    
    im = cv2.cvtColor(plt.imread(img_path), cv2.COLOR_BGR2HSV )
    rgn_im = cv2.cvtColor(plt.imread(img_path_rgn), cv2.COLOR_BGR2HSV )


    ax[0,i].imshow(im)
    ax[1,i].imshow(rgn_im)
    print(train_data.Label[idx[i].item()])

In [ ]:
#check if the dataset is balanced

plt.hist(train_data.Label)


In [ ]:
out = cv2.cvtColor(plt.imread(name2rgn(os.path.join(IMAGES,train_data.Image_id[idx[i].item()]))), cv2.COLOR_BGR2HSV )


In [ ]:
from torch import nn
from torch.utils.data import Dataset,DataLoader
import torchvision 
from torchvision import transforms
from sklearn.model_selection import StratifiedKFold
from numpy.ma.core import concatenate
import multiprocessing as mp
import ctypes

class chromatize_Transform(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self,y):
        y=y/y.sum(0)

        return y
    
class custom_dataset(Dataset):
    def __init__(self,table,image_path,type_='train',num_batch=50,transforms=transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    chromatize_Transform(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])):
        self.num_batch =num_batch
        self.name_table = table
        self.path = image_path
        self.transforms=transforms
        skf = StratifiedKFold(n_splits=num_batch,shuffle=True)

        self.mode = 0 # 0 RGB, 1 RGN, 2 both
        self.encode_dict = {'healthy':0,'brown':1,'blast':2}
        self.type=type_
        
        ##### caching to save time on dataloading 
        nb_samples= len(table)*224*224*3*2
        shared_array_base = mp.Array(ctypes.c_float, nb_samples)
        shared_array = np.ctypeslib.as_array(shared_array_base.get_obj())
        shared_array = shared_array.reshape(len(table),3,224,224,2)
        self.shared_array = torch.from_numpy(shared_array)
        self.use_cache = False
        
        
    def __getitem__(self,idx):
        
        conversion=cv2.COLOR_BGR2HSV
        
        path=os.path.join(self.path,self.name_table.Image_id[idx])
        
        if self.mode == 1:
            
            
            out = cv2.cvtColor(plt.imread(name2rgn(path)), conversion )
            out = Image.fromarray(out)
            out=self.transforms(out)
        
        elif self.mode == 0:
            out = cv2.cvtColor(plt.imread(path), conversion )
            out = Image.fromarray(out)
            out=self.transforms(out)
        
        else:
            if self.use_cache==False:
                
#                 rgn_im=cv2.cvtColor(plt.imread(name2rgn(path)), conversion )

#                 rgn_im = Image.fromarray(rgn_im)
                rgn_im=Image.open(name2rgn(path))
                tmp=rgn_im
                #rgn_im=rgn_im/rgn_im.sum()
                rgn_im=self.transforms(rgn_im).unsqueeze(-1)
#                 im=cv2.cvtColor(plt.imread(path), conversion )
#                 im = Image.fromarray(im)
                im=Image.open(path)
                im=self.transforms(im).unsqueeze(-1)
                
                out=torch.cat((im,rgn_im),dim=-1)
                
                
                #normalize to obtain chromacity
                self.shared_array[idx]=out
                
            else:
                out = self.shared_array[idx]
                
        if self.type=='test':
            return out 
        
        label =  self.encode_dict[self.name_table.Label[idx]]
        return out,label
    

    def set_mode(self,value):
        self.mode=value
    
    def set_use_cache(self,value):
        self.use_cache=value
    
    def __len__(self):
        
        return len(self.name_table)

In [ ]:
data = custom_dataset(train_data ,IMAGES )
data.set_mode(2)


In [ ]:
class classifier_model(nn.Module):
    def __init__(self,base,extract_feat):
        super(classifier_model,self).__init__()
        self.base = init_model(base,extarct_feat)
#         self.base.features.training =False
        self.dropout=nn.Dropout(0.5)
        self.fc=nn.Linear(2000,NUM_CLASSES)
        self.sigmoid = nn.Sigmoid() 
        
    def forward(self,y):
        y1=y[:,:,:,:,0].squeeze()
        y2=y[:,:,:,:,1].squeeze()        
        y=torch.cat((y1,y2),0)
        y=self.base(y)
        tmp=int(y.shape[0]/2)
        y1=y[:tmp,:]
        y2=y[tmp:,:]
        y=torch.cat((y1,y2),-1)
        y=self.sigmoid(self.fc(self.dropout(y)))
        return y

In [ ]:
train_dataloader.dataset.mode

In [ ]:
model=classifier_model(base,extarct_feat).to(device)
with torch.no_grad():
    print(model(training_data.to(device)).shape)

In [ ]:
import gc
gc.collect(generation=2)
gc.collect()
torch.cuda.empty_cache()

!nvidia-smi
print("memory reserved by caching allocator", torch.cuda.memory_reserved()/(1024*1024*1024))



In [ ]:
import torch
from sklearn.model_selection import StratifiedKFold

base="alex"

NUM_CLASSES=3    
NUM_FOLDS=4
NUM_EPOCHS = 10
extarct_feat=True
LR=1e-4

skf = StratifiedKFold(n_splits=NUM_FOLDS)
iterator = skf.split(train_data.Image_id, train_data.Label)
mem_track = mem_tracker()
mem_track.get_mem()

device=torch.device("cuda" if torch.cuda.is_available() else 'cpu') 
model=classifier_model(base,extarct_feat).to(device)
# model.classifier.requires_grad_(True)
model.requires_grad_(True)
# model =classifier_model(vgg_model).to(device)

optimiser= torch.optim.Adam(params2update(model,extarct_feat),LR)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimiser, gamma=0.99)
# from torchsummary import summary
# summary(model,(3,224,224))

mem_track.get_mem()

In [ ]:
import numpy as np

encode_dict = {'healthy':0,'brown':1,'blast':2}
counts=train_data.Label.value_counts()
weights=[0,0,0]
for key in list(encode_dict.keys()):
    weights[encode_dict[key]]=counts[key]

weights=1/np.array(weights)
weights=weights/weights.sum()
weights=torch.tensor(weights).to(device).float()

In [ ]:
def decay_lr(param,step,max_step, start_lr ,last_lr ):
    lr=((np.sqrt(max_step)*last_lr)/(start_lr*np.sqrt(3*step)))
    param["lr"]=lr


# steps=np.linspace(1,20*2,40)
# out=[decay_lr(step,40,1e-3,1e-6) for step in steps]
# plt.plot(out)

In [ ]:
# train_data=train_data.iloc[:200,:]

skf = StratifiedKFold(n_splits=NUM_FOLDS)
fold_iterator = skf.split(train_data.Image_id, train_data.Label)

# for train_index, val_index in iterator:
#     print(len(val_index)/(len(train_index)+len(val_index)))
#     print(len(train_index)/256)

In [ ]:
fold = 0
train_batch_size=256
for train_index, val_index in fold_iterator:
    print(f"starting fold {fold} ------------------------------ ")
    fold+=1
    train_dataset=custom_dataset(train_data.iloc[train_index,:].reset_index(drop=True),IMAGES)
    val_dataset = custom_dataset(train_data.iloc[val_index,:].reset_index(drop=True),IMAGES)
    val_dataloader=DataLoader(val_dataset,batch_size=32,num_workers=0)
    train_dataloader=DataLoader(train_dataset,batch_size=train_batch_size,num_workers=0)
    train_dataloader.dataset.set_mode(2)
    val_dataloader.dataset.set_mode(2)
    for epoch in range(NUM_EPOCHS):

        running_loss,iterator= 0,iter(train_dataloader)

        for i in range(len(iterator)):
            print(i)    
            training_data,training_lab=next(iterator)

            output = model(training_data.to(device))
            loss = nn.functional.cross_entropy(output,training_lab.to(device),weight=weights)
            
            running_loss+=loss

            loss.backward()
            
            if i%4==3:
                optimiser.step()
                optimiser.zero_grad()
                     
        train_dataloader.dataset.set_use_cache(True)   #use cache
            
        #validation run
        if epoch%6==5:
            
            val_loss,val_iterator = 0,iter(val_dataloader)
            with torch.no_grad():
                for k in range(len(val_dataloader)):                
                    val_data,val_lab = next(iter(val_dataloader))
                    val_output = model.eval()(val_data.to(device))
                    val_loss += nn.functional.cross_entropy(val_output,val_lab.to(device))    
            print(f'validation run ---- validation loss = {val_loss/k}')
            val_dataloader.dataset.set_use_cache(True)
            
            
        model.train()
        scheduler.step()
        running_loss=running_loss/i
        print(f'epoch[{epoch+1}]/[{NUM_EPOCHS}]  training loss --- {running_loss} --- lr-- {optimiser.param_groups[0]["lr"]}') 
        
    del train_dataloader
    del val_dataloader

In [ ]:
test_table = pd.read_csv(DATA_PATH+"/SampleSubmission.csv")
idx_to_drop=[]
for i,row in  enumerate(test_table.Image_id):
    if test_table.Image_id[i][-8:-4]=="_rgn":
        idx_to_drop.append(i)
        
test_table.drop(index=idx_to_drop, inplace=True)
test_table.reset_index(drop=True,inplace=True)

test_dataset=custom_dataset(test_table,IMAGES,type_='test')
test_dataloader = DataLoader(test_dataset,batch_size=128,shuffle=False) 
test_dataloader.dataset.set_mode(2)

In [ ]:
len(test_dataloader)

In [ ]:
test_iter=iter(test_dataloader)
test_output=torch.tensor([]).to(device)
for i,data in enumerate(test_iter):
    print(i)
    with torch.no_grad():
        test_output = torch.cat((test_output,model.eval()(data.to(device))),0)


In [ ]:
submission_table = pd.read_csv(DATA_PATH+"/SampleSubmission.csv")

encode_dict = {'healthy':0,'brown':1,'blast':2}
submission_table["blast"] = test_output[:,2].cpu()
submission_table["brown"] = test_output[:,1].cpu()
submission_table["healthy"] = test_output[:,0].cpu()
print(submission_table)

In [ ]:
np.where([test_table.Image_id=='id_zlht6gd0y2.jpg'])
min_=submission_table.iloc[:,1:].min(1)
# [=submission_table[name]-min_  ] 
for name in ['healthy','brown','blast']:
    submission_table[name]=submission_table[name]-min_


In [ ]:
submission_table.columns

In [ ]:
submission_table.to_csv("submission_b00ss3my.csv",index=False)
